### Imports

In [1]:
import numpy as np
import pandas as pd
import re

### Conversion/mapping functions

In [2]:
def convert_pressure_kpa_abs_to_kpa(val):
    return val - 101.3

def convert_afr_lambda_to_afr(val):
    return val * 14.7

def map_afr_diff(raw_val):
    return convert_afr_lambda_to_afr(raw_val / 1000)

def map_load(raw_val):
    return convert_pressure_kpa_abs_to_kpa(raw_val / 10)

def convert_temp_kelvin_to_celsius(val):
    return val - 273.15

def map_temp(raw_val):
    return convert_temp_kelvin_to_celsius(raw_val / 10)

def map_base_ignition(raw_val):
    return raw_val / 10 - 20

### Input function

In [3]:
#log_file_path = 'datalogs/20180713-mimos2home.csv'
    
def pandas_dataframe_from_datalog_csv(log_file_path):
    """
    Read from datalog csv file and return a pandas dataframe.
    """
    
    start_line=0
    content=""
    with open(log_file_path, 'r') as f:
        for line_num, line in enumerate(f):
    #        print(line_num, line)
            
            # Found log start line
            if line.find("Log : ") != -1:
    #            print(line.find("Log : "))
    #            print("break")
                start_line = line_num + 1
                print(start_line)
                break
            
            content = content + line
            
    channels = []
    channel_types = set()
    
    headers = ["Time"]
    pattern = r"Channel : (?P<channel>.+)\n*Type : (?P<type>.+)\n*DisplayMaxMin : (?P<max>.+),(?P<min>.+)\n*"
    for match in re.finditer(pattern, content):
        channel = match.group("channel")
        headers.append(channel)
        
        channel_type = match.group("type")
        channel_types.add(channel_type)
        
        val_max = match.group("max")
        val_min = match.group("min")
        
        channels.append({
                "Name": channel,
                "Type": channel_type,
                "Min": val_min,
                "Max": val_max
                })
    
        
    print("# channels:", len(headers) - 1)
    print("# channel types:", len(channel_types))      
    
    # Read csv log file
    
    df=pd.read_csv(log_file_path, skiprows=start_line, header=None)
#    df.head()
    
    # Rename headers
    df.columns = headers

    # Parse and set time as index
    df.index = pd.to_datetime(df.Time)
    df.set_index("Time", inplace=True)
    
    for channel in channels:
        channel_name = channel["Name"]
        channel_type = channel["Type"]
        val_min = channel["Min"]
        val_max = channel["Max"]
    
        if channel_type == "Percentage":
            df[channel_name] = df[channel_name] / 10
        elif channel_type == "Temperature":
            df[channel_name] = df[channel_name].apply(map_temp)
        elif channel_type == "Pressure":
            df[channel_name] = df[channel_name].apply(map_load)
        elif channel_type == "AFR":
            df[channel_name] = df[channel_name].apply(map_afr_diff)
        elif channel_type == "BatteryVoltage":
            df[channel_name] = df[channel_name] / 1000
        elif channel_type == "AngleIgnSprt2K":
            df[channel_name] = df[channel_name].apply(map_base_ignition)
        elif channel_type == "Angle":
            df[channel_name] = df[channel_name] / 10
        elif channel_type in ["Time_s", "EngineSpeed", "Raw"]:
            pass
        else:
            print("Unknown channel type:", channel_name, ":", channel_type)

    df.fillna(method='ffill', inplace=True)
    #df.dropna(axis=0, how='any', inplace=True

    return df

### Define data log paths and load data logs

In [4]:
LOG_FILE_PATHS = [
        'datalogs/20180716-home2mimos.csv',
        'datalogs/20180716-mimos2home.csv',
        
        'datalogs/20180717-home2mimos.csv',
        'datalogs/20180717-mimos2home.csv',
        
        'datalogs/20180718-home2mimos.csv',
        'datalogs/20180718-mimos2home.csv',
        
        'datalogs/20180719-home2mimos.csv',
        'datalogs/20180719-mimos2home.csv',
        
        'datalogs/20180720-home2mimos.csv',
        'datalogs/20180720-mimos2home.csv',
        ]
        
dfs=[]
for log_file_path in LOG_FILE_PATHS:
    df=pandas_dataframe_from_datalog_csv(log_file_path)
    dfs.append(df)

84
# channels: 26
# channel types: 14
Unknown channel type: FuelCoolantTempCorrection : Percentage1For1
Unknown channel type: IgnitionCoolantTempCorrection : AngleOffset10deg
Unknown channel type: TransientThrottleEnrichSensitivity : Time_us
Unknown channel type: TransientThrottleEnrichDecayRate : msPerEngCyl
84
# channels: 26
# channel types: 14
Unknown channel type: FuelCoolantTempCorrection : Percentage1For1
Unknown channel type: IgnitionCoolantTempCorrection : AngleOffset10deg
Unknown channel type: TransientThrottleEnrichSensitivity : Time_us
Unknown channel type: TransientThrottleEnrichDecayRate : msPerEngCyl
84
# channels: 26
# channel types: 14
Unknown channel type: FuelCoolantTempCorrection : Percentage1For1
Unknown channel type: IgnitionCoolantTempCorrection : AngleOffset10deg
Unknown channel type: TransientThrottleEnrichSensitivity : Time_us
Unknown channel type: TransientThrottleEnrichDecayRate : msPerEngCyl
84
# channels: 26
# channel types: 14
Unknown channel type: FuelCoo

In [5]:
len(dfs)
# Concat all the dataframes from different datalogs
#df=pd.concat([df for i, df in enumerate(dfs) if i < 7], axis=0)
df=pd.concat([df for i, df in enumerate(dfs)], axis=0)
df.reset_index(inplace=True)

### Define features and targets/labels

In [6]:
#features=['RPM', 'Load', 'TargetAFR', 'AFRDifference', 'AirTemp', 'CoolantTemp']
features=['RPM', 'Load', 'AFRDifference']
targets=['BaseFuel']

### Model parameters and training options

In [7]:
# Options
NUM_HIDDEN_LAYERS = 5
BATCH_SIZE = 100
VALIDATION_SPLIT = 0.3
NUM_EPOCHS = 200
HIDDEN_LAYER_NEURONS = 100
DROPOUT_RATE=0.2
OPTIMIZER = 'adam'
LOSS = 'mse'

### Define and train model

In [9]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler

x=df[features].values
y=df[targets].values

scaler=StandardScaler()
x=scaler.fit_transform(x)

scaler2=StandardScaler()
y=scaler2.fit_transform(y)

# Define DNN model
model = Sequential()
model.add(Dense(HIDDEN_LAYER_NEURONS, input_shape=(len(features),), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT_RATE))

for i in range(NUM_HIDDEN_LAYERS - 1):
    model.add(Dense(HIDDEN_LAYER_NEURONS, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(DROPOUT_RATE))

model.add(Dense(len(targets), activation='linear'))

checkpointer=ModelCheckpoint("models/model2-{epoch:02d}-{val_loss:.4f}.hdf5",
                             save_best_only=True,
                             verbose=1)
reduce_lr = ReduceLROnPlateau(patience=5,
                              verbose=1)

model.compile(optimizer=OPTIMIZER,
              loss=LOSS)

hist=model.fit(x, y,
               batch_size=BATCH_SIZE,
               shuffle=True,
               validation_split=VALIDATION_SPLIT,
               epochs=NUM_EPOCHS,
               callbacks=[checkpointer, reduce_lr,])

Train on 181896 samples, validate on 77956 samples
Epoch 1/200
181896/181896 [==============================] - 27s 147us/step - loss: 0.1244 - val_loss: 0.0101

Epoch 00001: val_loss improved from inf to 0.01006, saving model to models/model2-01-0.0101.hdf5
Epoch 2/200
181896/181896 [==============================] - 23s 125us/step - loss: 0.0385 - val_loss: 0.0088

Epoch 00002: val_loss improved from 0.01006 to 0.00878, saving model to models/model2-02-0.0088.hdf5
Epoch 3/200
181896/181896 [==============================] - 24s 131us/step - loss: 0.0320 - val_loss: 0.0085

Epoch 00003: val_loss improved from 0.00878 to 0.00852, saving model to models/model2-03-0.0085.hdf5
Epoch 4/200
181896/181896 [==============================] - 23s 124us/step - loss: 0.0283 - val_loss: 0.0061

Epoch 00004: val_loss improved from 0.00852 to 0.00613, saving model to models/model2-04-0.0061.hdf5
Epoch 5/200
181896/181896 [==============================] - 23s 126us/step - loss: 0.0265 - val_loss: 0.

181896/181896 [==============================] - 22s 118us/step - loss: 0.0167 - val_loss: 0.0019

Epoch 00046: val_loss did not improve from 0.00146

Epoch 00046: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-09.
Epoch 47/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0174 - val_loss: 0.0020

Epoch 00047: val_loss did not improve from 0.00146
Epoch 48/200
181896/181896 [==============================] - 22s 118us/step - loss: 0.0165 - val_loss: 0.0019

Epoch 00048: val_loss did not improve from 0.00146
Epoch 49/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0170 - val_loss: 0.0018

Epoch 00049: val_loss did not improve from 0.00146
Epoch 50/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0163 - val_loss: 0.0018

Epoch 00050: val_loss did not improve from 0.00146
Epoch 51/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0170 - val_loss: 0.0019

Epoch

181896/181896 [==============================] - 21s 116us/step - loss: 0.0166 - val_loss: 0.0019

Epoch 00092: val_loss did not improve from 0.00146
Epoch 93/200
181896/181896 [==============================] - 21s 115us/step - loss: 0.0171 - val_loss: 0.0019

Epoch 00093: val_loss did not improve from 0.00146
Epoch 94/200
181896/181896 [==============================] - 21s 116us/step - loss: 0.0172 - val_loss: 0.0022

Epoch 00094: val_loss did not improve from 0.00146
Epoch 95/200
181896/181896 [==============================] - 21s 115us/step - loss: 0.0164 - val_loss: 0.0026

Epoch 00095: val_loss did not improve from 0.00146
Epoch 96/200
181896/181896 [==============================] - 21s 115us/step - loss: 0.0163 - val_loss: 0.0018

Epoch 00096: val_loss did not improve from 0.00146

Epoch 00096: ReduceLROnPlateau reducing learning rate to 1.000000045813705e-19.
Epoch 97/200
181896/181896 [==============================] - 21s 116us/step - loss: 0.0169 - val_loss: 0.0019

Epoch


Epoch 00137: val_loss did not improve from 0.00146
Epoch 138/200
181896/181896 [==============================] - 22s 118us/step - loss: 0.0173 - val_loss: 0.0018

Epoch 00138: val_loss did not improve from 0.00146
Epoch 139/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0165 - val_loss: 0.0018

Epoch 00139: val_loss did not improve from 0.00146
Epoch 140/200
181896/181896 [==============================] - 23s 127us/step - loss: 0.0168 - val_loss: 0.0019

Epoch 00140: val_loss did not improve from 0.00146
Epoch 141/200
181896/181896 [==============================] - 22s 122us/step - loss: 0.0167 - val_loss: 0.0018

Epoch 00141: val_loss did not improve from 0.00146

Epoch 00141: ReduceLROnPlateau reducing learning rate to 1.0000001235416984e-28.
Epoch 142/200
181896/181896 [==============================] - 22s 120us/step - loss: 0.0167 - val_loss: 0.0018

Epoch 00142: val_loss did not improve from 0.00146
Epoch 143/200
181896/181896 [=================

181896/181896 [==============================] - 22s 119us/step - loss: 0.0162 - val_loss: 0.0019

Epoch 00183: val_loss did not improve from 0.00146
Epoch 184/200
181896/181896 [==============================] - 21s 118us/step - loss: 0.0170 - val_loss: 0.0019

Epoch 00184: val_loss did not improve from 0.00146
Epoch 185/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0169 - val_loss: 0.0018

Epoch 00185: val_loss did not improve from 0.00146
Epoch 186/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0174 - val_loss: 0.0018

Epoch 00186: val_loss did not improve from 0.00146

Epoch 00186: ReduceLROnPlateau reducing learning rate to 1.0000001256222317e-37.
Epoch 187/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0165 - val_loss: 0.0019

Epoch 00187: val_loss did not improve from 0.00146
Epoch 188/200
181896/181896 [==============================] - 22s 119us/step - loss: 0.0169 - val_loss: 0.0019


In [ ]:
def predict_base_fuel(rpm, load, afrdiff=0, targetafr=14.7, airtemp=60, coolanttemp=80,
                      model=model,
                      xscaler=scaler,
                      yscaler=scaler2):
#    x = np.array([rpm, load, targetafr, afrdiff, airtemp, coolanttemp])
    x = np.array([rpm, load, afrdiff])
    x = x.reshape(1, -1)
    x = xscaler.transform(x)
    y_predict = model.predict(x)
    y_predict = yscaler.inverse_transform(y_predict)
    return y_predict.item()

### Define fuel map RPMs and load points

In [ ]:
fuel_rpms=[10000, 8000, 7000, 6000, 5500, 5000, 4000, 3500, 3000, 2500, 2000, 1500, 1000, 500, 0]
fuel_loads=[-100, -85, -80, -70, -60, -50, -40, -30, -20, 0]

### Make prediction

In [18]:
fuel_rows=[]
for rpm in fuel_rpms:
    fuel_row = []
    
    for load in fuel_loads:    
        fuel_row.append(predict_base_fuel(rpm, load))
        
    fuel_rows.append(fuel_row)
    
    fuel_map=np.array(fuel_rows)
    
df_base_fuel_map_predict=pd.DataFrame(fuel_map)
df_base_fuel_map_predict.columns = fuel_loads
df_base_fuel_map_predict.index = fuel_rpms
df_base_fuel_map_predict

c:\users\xian_\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,-100,-85,-80,-70,-60,-50,-40,-30,-20,0
10000,95.156166,97.072845,98.251839,101.292374,104.738571,107.212234,109.492973,112.391235,114.940308,120.652802
8000,80.992577,80.656555,81.296867,83.045113,85.806847,89.005676,91.346283,94.262543,96.579269,102.227219
7000,72.939873,73.574600,73.921783,74.942772,76.689171,79.570923,82.304031,85.109131,87.276886,92.838837
6000,56.404900,66.701172,66.736687,67.701447,68.986969,70.598473,73.476135,75.995560,77.774910,82.798866
5500,36.528988,63.695129,63.519688,64.266251,65.331619,66.565918,68.902870,71.466042,73.106491,77.596542
5000,29.380203,59.135841,59.130615,60.118473,61.930611,62.509953,64.392212,66.825508,68.229317,73.133118
4000,25.597826,30.479658,32.840698,43.546482,48.089287,52.230488,53.142246,60.906494,61.734241,70.894989
3500,22.607437,30.184090,30.920879,41.313629,47.218407,51.852074,53.119232,59.029484,60.970016,71.511536
3000,17.353441,28.799351,30.064390,40.477009,47.875137,52.001602,57.249516,59.008663,63.138569,71.607170
2500,13.531326,23.367928,26.631954,39.987328,47.327034,53.208187,57.857674,59.643486,63.543201,71.594788
